<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Running_Flask_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls 'drive/My Drive/Colab Notebooks/Model'

In [5]:
import socket
print(socket.gethostbyname(socket.gethostname()))

172.28.0.2


In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask,render_template,jsonify,make_response,request
import threading

In [7]:
app= Flask(__name__)
padding_size=1000
model= tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/models/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/Colab Notebooks/models/sa_encoder.vocab")
print('Model and Vocabalory loaded.......')


@app.route("/")
def hello():
  return "I am alive"

def pad_size(vec,size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':6000}).start()



OSError: ignored

In [8]:
import requests
req = requests.get("http://172.28.0.2:6000/")
print(req.status_code)
print(req.text)

ConnectionError: ignored

In [9]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)


ConnectionError: ignored

In [10]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

ConnectionError: ignored

In [11]:
!ps -eaf | grep python

root          18       8  0 19:05 ?        00:00:02 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         124      18  0 19:07 ?        00:00:05 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-0b67caea-2354-432a-b843-16a4ba6370b8.json
root         513     124  0 19:29 ?        00:00:00 /bin/bash -c ps -eaf | grep python
root         515     513  0 19:29 ?        00:00:00 grep python


In [12]:
import flask
flask.__version__

'1.1.2'

In [13]:
!pip install flask-ngrok

In [14]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/models/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/Colab Notebooks/models/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

if __name__ == '__main__':
    app.run()

OSError: ignored